In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from keras import models
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras import regularizers

In [2]:
x_test = pd.read_excel('C:\\Users\\Daniel\\Documents\\Flatiron\\Capstone\\sports_predictions\\stats.xlsx')

In [3]:
x_test

,Home Points For,Home Points Against,Home eFG%,Home FTr,Home ORB%,Home DRB%,Home AST%,Home STL%,Home BLK%,Home TOV%,...,Away DRB%,Away AST%,Away STL%,Away BLK%,Away TOV%,Away ORTG,Away DRtg,Away GP,Away Win Rate,Vegas Prediction
0,103.5,101.3,0.522,0.236,23.7,73.6,63.2,8.1,3.8,14.9,...,77.3,56.3,8.2,6.3,12.8,106.4,105.7,13,0.500000,-2.5
1,97.6,107.3,0.468,0.242,21.0,75.7,61.2,8.6,5.2,14.0,...,75.7,55.9,6.8,5.2,12.6,109.3,110.0,13,0.428571,-4.5
2,108.7,103.8,0.523,0.237,20.1,78.6,57.7,7.0,5.9,12.9,...,78.0,62.9,7.4,4.5,13.9,104.2,114.0,14,0.142857,4.5
3,107.5,110.1,0.509,0.202,25.2,80.4,55.0,6.4,3.6,11.4,...,76.3,58.7,6.5,5.2,14.6,100.6,110.4,13,0.330000,10.5
4,109.6,109.2,0.525,0.240,23.2,76.4,54.9,7.5,6.2,12.2,...,77.5,58.0,7.1,6.6,14.3,109.6,108.9,14,0.250000,3.5
5,104.5,106.1,0.510,0.233,21.3,80.6,56.0,7.1,4.8,11.5,...,79.7,59.8,8.4,4.2,13.3,107.6,101.6,13,0.400000,4.5
6,107.8,114.4,0.498,0.189,27.6,78.9,59.7,9.8,5.9,12.1,...,76.7,55.0,8.0,4.5,16.8,99.4,108.4,13,0.000000,11.0
7,104.6,109.0,0.489,0.185,26.2,69.9,61.6,9.5,5.8,14.2,...,77.6,62.1,9.8,5.1,12.0,109.9,104.3,12,0.800000,-4.0
8,99.3,107.5,0.474,0.223,22.1,76.2,53.9,7.3,4.1,13.3,...,80.3,58.9,7.3,2.4,13.4,111.8,105.2,13,0.500000,-9.0
9,108.6,109.0,0.528,0.209,23.4,80.4,58.1,8.1,4.6,12.5,...,77.0,51.4,10.5,4.4,11.7,109.6,108.4,14,0.833333,1.5


In [4]:
df = pd.read_csv('avg_by_game')
odds = pd.read_csv('nba_game_odds')

In [5]:
data = df.set_index(df['Unnamed: 0'], drop=True)
data.index.rename('GameID', inplace=True)
data.drop(columns='Unnamed: 0', inplace=True)

odds.set_index(odds['GameID'], drop=True, inplace=True)

In [6]:
data.sort_index(inplace=True)
odds.sort_index(inplace=True)
data['Home Wins By (Vegas)'] = odds['Home Win By (Vegas)']

In [7]:
#some misalignments/buggy odds
data.dropna(inplace=True)
data.drop(index=data[data['Home Wins By (Vegas)']>30].index, inplace=True)

In [8]:
results = pd.DataFrame(columns=['linreg', 'forest', 'gb', 'NN', 'Vegas Prediction'],
                      index = range(len(x_test)))

In [9]:
x = data.drop(columns='Home Win By')
y = data['Home Win By']

In [10]:
linreg = LinearRegression()
forest = RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3, n_estimators=200)
gb = GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5, min_samples_split=3, n_estimators=200, random_state=42)
ml_models = [linreg, forest, gb]

In [11]:
def run_model(x, y, x_test, model):

    model.fit(x, y)
    y_test_pred = model.predict(x_test)
    
    return y_test_pred

In [12]:
nn = models.Sequential()
nn.add(Dense(500, input_dim=29, activation= 'relu'))
nn.add(Dense(250, activation= 'relu'))
nn.add(Dense(125, activation= 'relu'))

nn.add(Dense(1, activation='linear'))

nn.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

In [13]:
nn.fit(x, y, epochs=200, verbose=2, batch_size=300)
test_preds = nn.predict(x_test)
results['NN'] = test_preds

Epoch 1/200
36/36 - 0s - loss: 355.8458 - mean_squared_error: 355.8458
Epoch 2/200
36/36 - 0s - loss: 163.5410 - mean_squared_error: 163.5410
Epoch 3/200
36/36 - 0s - loss: 141.8592 - mean_squared_error: 141.8592
Epoch 4/200
36/36 - 0s - loss: 134.9940 - mean_squared_error: 134.9940
Epoch 5/200
36/36 - 0s - loss: 135.6093 - mean_squared_error: 135.6093
Epoch 6/200
36/36 - 0s - loss: 132.6811 - mean_squared_error: 132.6811
Epoch 7/200
36/36 - 0s - loss: 133.1906 - mean_squared_error: 133.1906
Epoch 8/200
36/36 - 0s - loss: 137.6920 - mean_squared_error: 137.6920
Epoch 9/200
36/36 - 0s - loss: 131.8901 - mean_squared_error: 131.8901
Epoch 10/200
36/36 - 0s - loss: 130.2010 - mean_squared_error: 130.2010
Epoch 11/200
36/36 - 0s - loss: 129.5038 - mean_squared_error: 129.5038
Epoch 12/200
36/36 - 0s - loss: 131.4211 - mean_squared_error: 131.4211
Epoch 13/200
36/36 - 0s - loss: 129.8716 - mean_squared_error: 129.8716
Epoch 14/200
36/36 - 0s - loss: 128.6199 - mean_squared_error: 128.6199
E

Epoch 115/200
36/36 - 0s - loss: 125.5646 - mean_squared_error: 125.5646
Epoch 116/200
36/36 - 0s - loss: 125.1542 - mean_squared_error: 125.1542
Epoch 117/200
36/36 - 0s - loss: 128.2842 - mean_squared_error: 128.2842
Epoch 118/200
36/36 - 0s - loss: 126.5168 - mean_squared_error: 126.5168
Epoch 119/200
36/36 - 0s - loss: 126.4107 - mean_squared_error: 126.4107
Epoch 120/200
36/36 - 0s - loss: 125.9472 - mean_squared_error: 125.9472
Epoch 121/200
36/36 - 0s - loss: 127.6139 - mean_squared_error: 127.6139
Epoch 122/200
36/36 - 0s - loss: 127.6302 - mean_squared_error: 127.6302
Epoch 123/200
36/36 - 0s - loss: 127.2809 - mean_squared_error: 127.2809
Epoch 124/200
36/36 - 0s - loss: 125.7595 - mean_squared_error: 125.7595
Epoch 125/200
36/36 - 0s - loss: 125.0838 - mean_squared_error: 125.0838
Epoch 126/200
36/36 - 0s - loss: 125.3085 - mean_squared_error: 125.3085
Epoch 127/200
36/36 - 0s - loss: 125.0225 - mean_squared_error: 125.0225
Epoch 128/200
36/36 - 0s - loss: 125.6119 - mean_sq

In [14]:
column_names = ['linreg', 'forest', 'gb', 'NN']

In [15]:
for i, model in enumerate(ml_models):
    preds = run_model(x, y, x_test, model)
    results[column_names[i]] = preds

In [16]:
results['Vegas Prediction'] = x_test['Vegas Prediction']

In [17]:
results

,linreg,forest,gb,NN,Vegas Prediction
0,-1.293373,0.937006,-2.170528,0.995070,-2.5
1,-8.443143,-6.908248,-8.248022,-10.680583,-4.5
2,8.714518,9.621085,13.773533,10.184610,4.5
3,5.081158,8.087742,8.547101,4.336245,10.5
4,-1.569947,0.789143,-2.054795,-1.793739,3.5
5,-6.706326,5.287300,0.746238,-6.573121,4.5
6,2.176824,7.783033,7.967278,1.860505,11.0
7,-9.176897,-10.194078,-12.237677,-11.700897,-4.0
8,-14.408325,-8.049083,-9.112840,-15.928317,-9.0
9,-1.825068,-0.657029,0.544782,-1.700314,1.5
